In [1]:
# favorite example

# cam2, start_index = 20, end_index = 23

# point №40
# it corresponds to instance 53 (image 20)
# it corresponds to instance 45 (image 21)
# it corresponds to instance 53 (image 22)

In [2]:
from pathlib import Path

from pcd_dataset.kitti_dataset import KittiDataset

dataset_path = 'dataset/'
sequence = '00'
sam_labels_path = Path.cwd().joinpath('pipeline', 'vfm-labels', 'sam', sequence)

kitti = KittiDataset(dataset_path, sequence, sam_labels_path)

In [3]:
from mapping.simple_mapping import SimpleMapping

sm = SimpleMapping(kitti)
coo_matrix, pcds = sm.get_combined_labeled_point_clouds('cam2', 20, 23)

INFO:simple_mapping.SimpleMapping:Image 20 processed
INFO:simple_mapping.SimpleMapping:Image 21 processed
INFO:simple_mapping.SimpleMapping:Image 22 processed


In [5]:
pcds[0]

PointCloud with 277369 points.

In [6]:
pcds[2] + pcds[1] + pcds[0]

PointCloud with 832606 points.

In [4]:
from mapping.sparse_matrix_utils import construct_coo_matrix_for_multiple_views

coo_result = construct_coo_matrix_for_multiple_views(coo_matrix, 20)

In [5]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse_output=True)

In [6]:
enc.fit((coo_result.T).toarray())

OneHotEncoder(handle_unknown='ignore')

In [7]:
enc.categories_

[array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10., 11., 12., 13.,
        14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.,
        27., 28., 29., 30., 31., 32., 33., 34., 35., 37., 38., 39., 40.,
        42., 43., 44., 45., 46., 48., 49., 50., 51., 53., 54., 56., 57.,
        58., 59., 62., 63., 64., 67., 68., 69., 71., 73., 74., 75., 76.,
        77., 78., 79., 80.]),
 array([ 0.,  2.,  3.,  4.,  5.,  6.,  7., 10., 11., 12., 13., 14., 15.,
        16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27., 28.,
        29., 30., 31., 32., 33., 34., 35., 36., 37., 39., 40., 41., 42.,
        43., 44., 45., 47., 49., 50., 51., 52., 54., 55., 56., 58., 59.,
        60., 62., 64., 65., 67., 69., 71., 72., 73., 74.]),
 array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
        13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
        26., 27., 28., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39.,
        40., 41., 42., 43., 44., 4

In [8]:
s = enc.transform(coo_result.T.toarray()).toarray()

In [9]:
for itr in range(len(s[40])):
    if s[40, itr] != 0.0:
        print(itr)

48
110
182


In [41]:
p = enc.inverse_transform(s)

k = 0
for itr in range(len(p)):
    if (p[itr] == [0.0, 0.0, 0.0]).all():
        k = k + 1
        
print(len(p) - k)

92411


In [46]:
import numpy as np

p = enc.inverse_transform(s)
points = np.zeros((len(p) - k), dtype=int)

map = []

k = 0
i = 0
for itr in range(len(p)):
    if (p[itr] == [0.0, 0.0, 0.0]).all():
        k = k + 1
    else:
        if k < 10:
            print(itr)
        points[i] = itr 
        map.append({'old_pcd': itr, 'new_pcd': i})
        i = i + 1

0
1
2
3
4
5


In [53]:
map[117]

{'old_pcd': 1171, 'new_pcd': 117}

In [44]:
map[3]

{3}

In [ ]:
map

In [35]:
p[5]

array([ 0.,  0., 61.])

In [38]:
map[6]

{6, 20}

In [21]:
len(points)

92411

In [22]:
k + len(points)

832606

In [23]:
len(p)

832606

In [54]:
import open3d as o3d

pcd_baby = o3d.geometry.PointCloud()
pcd_baby.points = o3d.utility.Vector3dVector(points)

RuntimeError: Unable to cast Python instance to C++ type (compile in debug mode for details)

In [11]:
import numpy as np
len(np.asarray(pcds[0].points))

277369

In [12]:
import utils.pcd_utils
pcd = utils.pcd_utils.visualize_from_one_hot_encoding(s, enc, pcds, 20, 20)

[53. 45. 53.]
53.0
832606
PointCloud with 277369 points.
277369


In [12]:
import open3d as o3d

o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [15]:
i = enc.inverse_transform(s)

start = 20
end = 46
for point in range(start, end) :
    print("{} : {}".format(point, i[point]))

20 : [15.  0.  0.]
21 : [15. 17.  0.]
22 : [15. 17. 15.]
23 : [15. 17. 15.]
24 : [15. 17. 15.]
25 : [15. 17. 15.]
26 : [15. 17. 15.]
27 : [15. 17. 15.]
28 : [ 0.  0. 15.]
29 : [0. 0. 0.]
30 : [0. 0. 0.]
31 : [0. 0. 0.]
32 : [53. 45.  0.]
33 : [53. 45.  0.]
34 : [53. 45. 53.]
35 : [53. 45. 53.]
36 : [53. 45. 53.]
37 : [53. 45. 53.]
38 : [53. 45. 53.]
39 : [53. 45. 53.]
40 : [53. 45. 53.]
41 : [53. 45. 53.]
42 : [53. 45. 53.]
43 : [53. 45. 53.]
44 : [53. 45. 53.]
45 : [ 7.  4. 53.]


In [9]:
from sklearn.metrics.pairwise import pairwise_distances
dist555 = pairwise_distances(s[0:450])

In [18]:
dist555[26, 37]

2.449489742783178

In [27]:
dist555.shape

(450, 450)

In [11]:
dist = np.exp(dist555)

In [14]:
dist555

array([[0.        , 0.        , 0.        , ..., 1.41421356, 1.41421356,
        1.41421356],
       [0.        , 0.        , 0.        , ..., 1.41421356, 1.41421356,
        1.41421356],
       [0.        , 0.        , 0.        , ..., 1.41421356, 1.41421356,
        1.41421356],
       ...,
       [1.41421356, 1.41421356, 1.41421356, ..., 0.        , 0.        ,
        0.        ],
       [1.41421356, 1.41421356, 1.41421356, ..., 0.        , 0.        ,
        0.        ],
       [1.41421356, 1.41421356, 1.41421356, ..., 0.        , 0.        ,
        0.        ]])

In [13]:
dist

array([[1.        , 1.        , 1.        , ..., 4.11325038, 4.11325038,
        4.11325038],
       [1.        , 1.        , 1.        , ..., 4.11325038, 4.11325038,
        4.11325038],
       [1.        , 1.        , 1.        , ..., 4.11325038, 4.11325038,
        4.11325038],
       ...,
       [4.11325038, 4.11325038, 4.11325038, ..., 1.        , 1.        ,
        1.        ],
       [4.11325038, 4.11325038, 4.11325038, ..., 1.        , 1.        ,
        1.        ],
       [4.11325038, 4.11325038, 4.11325038, ..., 1.        , 1.        ,
        1.        ]])

In [10]:
import numpy as np
labels = np.arange(450)
# print(labels)

from awesome_normalized_cut import normalized_cut

u = normalized_cut(dist555, labels, 10)

(450, 450)
1.0053009775053847
ggg
(449, 449)
1.005379397149197
ggg
(448, 448)
1.0054598190116366
ggg
(447, 447)
1.0055423207654997
ggg
(446, 446)
0.9980425595643315
ggg
(263, 263)
1.009400075239312
ggg
(262, 262)
1.009647154489983
ggg
(261, 261)
1.009783896937795
ggg
(260, 260)
0.9967122979374823
ggg
(184, 184)
0.9997205585836595
ggg
(88, 88)
1.0071310772508884
ggg
(38, 38)
0.9844930755037444
ggg
(17, 17)
0.995698439912337
ggg
(8, 8)
inf
(9, 9)
1.0121062129975167
ggg
(7, 7)
1.196390590523579
ggg
(6, 6)
0.8989754991058333
ggg
(5, 5)
0.9313211564230552
ggg
(4, 4)
1.0444630713806553
ggg
(2, 2)
(2, 2)
(1, 1)
(1, 1)
(1, 1)
(2, 2)
(21, 21)
1.119534186861145
ggg
(20, 20)
0.9068235153789725
ggg
(19, 19)
1.1748701714947323
ggg
(18, 18)
0.8586719040159165
ggg
(17, 17)
0.9386491905439174
ggg
(7, 7)
0.7844543145495974
ggg
(6, 6)
0.8006715770097076
ggg
(5, 5)
inf
(1, 1)
(1, 1)
(10, 10)
0.7584714357839281
ggg
(9, 9)
0.7648486160294723
ggg
(8, 8)
0.7731627476097037
ggg
(7, 7)
0.7844543145495974
ggg
(

/Users/sofiavivdich/projects/lidar-labelling/venv/lib/python3.9/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:939: LinAlgWarning: Diagonal number 8 is exactly zero. Singular matrix.
  self.M_lu = lu_factor(M)
/Users/sofiavivdich/projects/lidar-labelling/awesome_normalized_cut.py:13: RuntimeWarning: invalid value encountered in scalar divide
  return (cost / assoc_a) + (cost / assoc_b)
/Users/sofiavivdich/projects/lidar-labelling/venv/lib/python3.9/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:939: LinAlgWarning: Diagonal number 5 is exactly zero. Singular matrix.
  self.M_lu = lu_factor(M)
/Users/sofiavivdich/projects/lidar-labelling/venv/lib/python3.9/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:939: LinAlgWarning: Diagonal number 4 is exactly zero. Singular matrix.
  self.M_lu = lu_factor(M)
/Users/sofiavivdich/projects/lidar-labelling/venv/lib/python3.9/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:939: LinAlgWarning: Diagonal number 

0.9736477934814642
ggg
(36, 36)
1.1100652638326358
ggg
(1, 1)
(35, 35)
0.9787100354110722
ggg
(29, 29)
0.9399318112777375
ggg
(12, 12)
1.6743166898287232
ggg
(11, 11)
inf
(1, 1)
(17, 17)
1.0360683464838063
ggg
(5, 5)
inf
(12, 12)
0.6205349932299216
ggg
(11, 11)
0.6241431327656015
ggg
(10, 10)
0.6285873992740718
ggg
(9, 9)
0.6341965192357523
ggg
(1, 1)
(8, 8)
0.64149752781164
ggg
(7, 7)
1.4803599393981928
ggg
(6, 6)
inf
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(6, 6)
1.074855006934422
ggg
(3, 3)
inf
(3, 3)
inf
(14, 14)
1.000614153374794
ggg
(4, 4)
inf
(10, 10)
0.7584714357839281
ggg
(9, 9)
0.7648486160294723
ggg
(8, 8)
0.8485152435769412
ggg
(2, 2)
(6, 6)
0.8006715770097076
ggg
(5, 5)
inf
(1, 1)
(1, 1)
(1, 1)
(96, 96)
0.9842681735040507
ggg
(95, 95)
1.0262373925364385
ggg
(94, 94)
1.0279899690096304
ggg
(1, 1)
(93, 93)
0.9818534271117002
ggg
(92, 92)
1.000943104201987
ggg
(55, 55)
0.9652946939087242
ggg
(54, 54)
0.9655019328791128
ggg
(53, 53)
0.9657179797892484
ggg
(52, 52)
1.057369264512968

/Users/sofiavivdich/projects/lidar-labelling/venv/lib/python3.9/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:939: LinAlgWarning: Diagonal number 6 is exactly zero. Singular matrix.
  self.M_lu = lu_factor(M)
/Users/sofiavivdich/projects/lidar-labelling/venv/lib/python3.9/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:939: LinAlgWarning: Diagonal number 9 is exactly zero. Singular matrix.
  self.M_lu = lu_factor(M)
/Users/sofiavivdich/projects/lidar-labelling/venv/lib/python3.9/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:939: LinAlgWarning: Diagonal number 10 is exactly zero. Singular matrix.
  self.M_lu = lu_factor(M)


0.9733254739717424
ggg
(20, 20)
0.9363321077110955
ggg
(13, 13)
0.9376063981022471
ggg
(7, 7)
0.7844543145495978
ggg
(6, 6)
inf
(1, 1)
(6, 6)
0.8006715770097076
ggg
(5, 5)
1.6174919570103885
ggg
(4, 4)
inf
(1, 1)
(1, 1)
(7, 7)
0.7844543145495974
ggg
(6, 6)
0.8006715770097076
ggg
(5, 5)
inf
(1, 1)
(1, 1)
(22, 22)
0.9563581680649113
ggg
(10, 10)
0.9308272505765144
ggg
(6, 6)
0.8006715770097076
ggg
(5, 5)
inf
(1, 1)
(4, 4)
0.7262098355020445
ggg
(3, 3)
0.8123273572878915
ggg
(2, 2)
(1, 1)
(1, 1)
(12, 12)
0.9934248294756187
ggg
(8, 8)
inf
(4, 4)
inf
(1, 1)
(46, 46)
0.9846714097823096
ggg
(18, 18)
1.1790736519473883
ggg
(17, 17)
1.0012944994843176
ggg
(13, 13)
0.7459456564224412
ggg
(12, 12)
0.7493319971183675
ggg
(11, 11)
0.7534249471704547
ggg
(10, 10)
0.7584714357839281
ggg
(9, 9)
0.7648486160294723
ggg
(8, 8)
0.7731627476097037
ggg
(7, 7)
0.7844543145495974
ggg
(6, 6)
0.8006715770097076
ggg
(5, 5)
inf
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(4, 4)
inf
(1, 1)
(28, 28)
1.1

/Users/sofiavivdich/projects/lidar-labelling/venv/lib/python3.9/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:939: LinAlgWarning: Diagonal number 12 is exactly zero. Singular matrix.
  self.M_lu = lu_factor(M)


In [11]:
u

[array([ 21, 106, 122, 126, 144, 239, 435, 449]),
 array([ 28, 311]),
 array([ 57, 398]),
 array([211]),
 array([332]),
 array([74]),
 array([133, 366]),
 array([ 50, 242, 268, 365, 387]),
 array([357]),
 array([264]),
 array([ 63, 135, 164, 377]),
 array([174, 420]),
 array([416]),
 array([276]),
 array([341]),
 array([134]),
 array([388]),
 array([69]),
 array([289]),
 array([71]),
 array([65]),
 array([  6, 156, 170, 203, 220, 223, 285, 347, 406, 425, 429]),
 array([41]),
 array([ 68, 182, 231, 280, 381]),
 array([209]),
 array([175, 191, 224, 329, 334, 408]),
 array([115]),
 array([313]),
 array([410]),
 array([200]),
 array([116]),
 array([ 76, 192, 363]),
 array([376, 399, 400]),
 array([ 78, 217, 315, 383]),
 array([227, 236]),
 array([ 99, 162, 270, 303, 328]),
 array([419]),
 array([243]),
 array([290]),
 array([67]),
 array([87]),
 array([  5, 128]),
 array([177]),
 array([322]),
 array([369]),
 array([183]),
 array([235]),
 array([318]),
 array([160, 308]),
 array([ 20, 439]

In [12]:
# 1) выбросить ненужные точки. создать новое облако и мапу старое - новое
# 2) voxel_down_sample применить к маленькому облаку
# применить к нему граф кат, раскрасить по кластерам ИСХОДНОЕ маленькое облако с новыми инстансами
# 3) kd деревья почитать чтобы найти соседей

In [13]:
u

[array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
         40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
         53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
         66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
         79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
         92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
        105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
        118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
        131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
        144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
        157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
        170, 171, 172, 173, 174, 175, 176, 177, 178

In [ ]:
import scipy.spatial
res = scipy.spatial.distance.cdist(s, s, 'euclidean')

In [1]:
res

NameError: name 'res' is not defined

In [ ]:
# найти функцию которая на разреженных матрицах считает расстояния
# на каком-то этапе надо выкинуть точки которые не попадают ни в один снимок